In [1]:
# import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import (confusion_matrix,
                             ConfusionMatrixDisplay,
                             classification_report,
                             RocCurveDisplay,
                             roc_curve,
                             r2_score,
                             mean_squared_error,
                             auc, accuracy_score)
from sklearn.feature_selection import SequentialFeatureSelector
import tqdm
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from scipy.io.arff import loadarff

## Pre-settings

Carrega base:

In [3]:
df_data = pd.DataFrame(loadarff(r'C:\Users\luiz_\Desktop\MLI - Projeto\MLI - Projeto\dataset_6_letter.arff')[0])
df_data['class'] = df_data['class'].astype(str)

In [5]:
df_data.head()

,x-box,y-box,width,high,onpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybr,xy2br,x-ege,xegvy,y-ege,yegvx,class
0,2.0,4.0,4.0,3.0,2.0,7.0,8.0,2.0,9.0,11.0,7.0,7.0,1.0,8.0,5.0,6.0,Z
1,4.0,7.0,5.0,5.0,5.0,5.0,9.0,6.0,4.0,8.0,7.0,9.0,2.0,9.0,7.0,10.0,P
2,7.0,10.0,8.0,7.0,4.0,8.0,8.0,5.0,10.0,11.0,2.0,8.0,2.0,5.0,5.0,10.0,S
3,4.0,9.0,5.0,7.0,4.0,7.0,7.0,13.0,1.0,7.0,6.0,8.0,3.0,8.0,0.0,8.0,H
4,6.0,7.0,8.0,5.0,4.0,7.0,6.0,3.0,7.0,10.0,7.0,9.0,3.0,8.0,3.0,7.0,H


Converte coluna de classificação de categórica para numérica:

In [6]:
# Mapeando os valores da classe para inteiro (para fins de visualização da região de decisão)
df_data['class'] = pd.factorize(df_data['class'])[0]

Separa colunas de features (X) e coluna de classificação (y):

In [7]:
# Features
X = df_data.iloc[:,0:-1]
# Class
y = df_data.iloc[:,-1]

Divide conjunto de treinamento  (80%) e conjunto de teste (20%):

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [9]:
# models
models_functions = [
    ("Decision Tree",DecisionTreeClassifier(criterion='log_loss',max_depth=100,splitter='best')),
    ("KNN",KNeighborsClassifier(n_neighbors=1,metric='euclidean')),
    ("Random Forest", RandomForestClassifier(criterion='entropy',max_depth=1000,n_estimators=10)),
    ("Logistic Regression", LogisticRegression(penalty='l2',C=0.5,solver='newton-cg')),
    ("Gaussian", GaussianNB()),
    ("MLP", MLPClassifier(activation='tanh', hidden_layer_sizes= (16, 26), learning_rate= 'invscaling')),
    ("SVM", SVC(C= 20, decision_function_shape= 'ovo', kernel= 'rbf'))
]

# SequentialFeatureSelection - Oficial

In [10]:
for models in tqdm.tqdm(models_functions):

    for n_features in range(8,13,4):

        start = time.time()
        name = models[0]
        model = models[1]

        # Nome do classificador
        print("*"*(len(name)+2))
        print(f"*{name} with - {n_features} features*")
        print("*"*(len(name)+2))
        print("-"*50)


        kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=199)
        sfs = SequentialFeatureSelector(model, n_features_to_select=n_features, direction='forward', cv=10)
        sfs.fit(X, y)

        best_features_cv = sfs.get_feature_names_out()
        print(f"Best features to {name} with {n_features}: {best_features_cv}")

        # Seleção apenas das melhores features
        X_best_cv = df_data[best_features_cv]

        X_train, X_test, y_train, y_test = train_test_split(X_best_cv, y, test_size=0.2, stratify=y)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        print()
        print(classification_report(y_test, y_pred))
        print()

        print(f"MSE to {name} with - {n_features} features:", mean_squared_error(y_test, y_pred))
        print(f"R2_score {name} with - {n_features} features:", r2_score(y_test, y_pred))
        print(f"Accuracy score {name} with - {n_features} features:", accuracy_score(y_test,y_pred))
        print()

        # print(f"Confusion Matrix to {name} with - {n_features} features:")
        # ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test, y_pred, labels=df_data['class'].unique()),display_labels=df_data['class'].unique()).plot()
        # print()

        end = time.time()

        print(f"Execution time to {name}: {(end-start):.2f}s")


  0%|          | 0/1 [00:00<?, ?it/s]

*****
*MLP with - 8 features*
*****
--------------------------------------------------
Best features to MLP with 8: ['x2bar' 'y2bar' 'xybar' 'x2ybr' 'xy2br' 'x-ege' 'xegvy' 'y-ege']

              precision    recall  f1-score   support

           0       0.77      0.85      0.81       147
           1       0.85      0.84      0.85       161
           2       0.69      0.56      0.62       150
           3       0.68      0.67      0.68       147
           4       0.84      0.73      0.78       155
           5       0.85      0.86      0.86       157
           6       0.66      0.75      0.70       151
           7       0.94      0.92      0.93       158
           8       0.79      0.74      0.76       161
           9       0.77      0.84      0.80       153
          10       0.90      0.92      0.91       158
          11       0.85      0.76      0.80       148
          12       0.84      0.73      0.78       154
          13       0.68      0.75      0.71       150
      

100%|██████████| 1/1 [8:37:51<00:00, 31071.46s/it]


              precision    recall  f1-score   support

           0       0.93      0.82      0.87       147
           1       0.88      0.80      0.83       161
           2       0.77      0.78      0.77       149
           3       0.68      0.76      0.72       147
           4       0.84      0.81      0.83       155
           5       0.88      0.89      0.88       157
           6       0.79      0.74      0.76       152
           7       0.86      0.91      0.88       158
           8       0.79      0.83      0.81       161
           9       0.88      0.89      0.88       153
          10       0.92      0.92      0.92       158
          11       0.85      0.82      0.84       148
          12       0.78      0.80      0.79       154
          13       0.81      0.83      0.82       150
          14       0.81      0.87      0.84       157
          15       0.93      0.88      0.90       152
          16       0.85      0.89      0.87       157
          17       0.89   